# Test conversions

- Quaternion <-> Euler angles
- Matrix <-> Euler Angles
- Quaternion <-> Matrix

In [23]:
from numba import cuda
cuda.select_device(0)
cuda.close()

In [1]:
import sys
sys.path.append("../..") 
from cryoem.conversions import *
from scipy.spatial.transform import Rotation as R

In [2]:
import scipy
scipy.__version__

'1.4.0'

In [3]:
# Non-tf code for double-check
r = R.from_euler('zyz', [0.0, 0.0, 0.0])
r.as_quat()

array([0., 0., 0., 1.])

In [4]:
# Euler to Quaternions conversion
assert np.all(np.isclose(euler2quaternion([[0.0, 0.0, 0.0]]).numpy(), [0.0, 0.0, 0.0, 1.0], atol=1e-5)), euler2quaternion([[0.0, 0.0, 0.0]]).numpy()
assert np.all(np.isclose(euler2quaternion([[np.pi, 0.0, 0.0]]).numpy(), [0.0, 0.0, 1.0, 0.0], atol=1e-5)), euler2quaternion([[np.pi, 0.0, 0.0]]).numpy()
assert np.all(np.isclose(euler2quaternion([[0.0, np.pi, 0.0]]).numpy(), [0.0, 1.0, 0.0, 0.0], atol=1e-5)), euler2quaternion([[0.0, np.pi, 0.0]]).numpy()
assert np.all(np.isclose(euler2quaternion([[0.0, 0.0, np.pi]]).numpy(), [0.0, 0.0, 1.0, 0.0], atol=1e-5)), euler2quaternion([[0.0, 0.0, np.pi]]).numpy()
assert np.all(np.isclose(euler2quaternion([[np.pi, 0.0, np.pi]]).numpy(), [0.0, 0.0, 0.0, 1.0], atol=1e-5)), euler2quaternion([[np.pi, 0.0, np.pi]]).numpy()
assert np.all(np.isclose(euler2quaternion([[np.pi, np.pi, np.pi]]).numpy(), [0.0, 1.0, 0.0, 0.0], atol=1e-5)), euler2quaternion([[np.pi, np.pi, np.pi]]).numpy()

In [5]:
r = R.from_quat([0, 0, 0, 1])
r.as_euler('zyz', degrees=True)

/home/jbanjac/anaconda3/envs/protein_reconstruction/lib/python3.6/site-packages/scipy/spatial/transform/rotation.py:135: UserWarning: Gimbal lock detected. Setting third angle to zero since it is not possible to uniquely determine all angles.
  warnings.warn("Gimbal lock detected. Setting third angle to zero since"


array([0., 0., 0.])

In [6]:
# print(angles_true[0])
# e = angles_true[0]
# q = euler2quaternion([e])
# e_new = quaternion2euler([q])
# e_new.numpy()

In [7]:
# d_q(q, euler2quaternion(e_new)).numpy()

In [8]:
assert np.all(np.isclose(quaternion2euler([[0.0, 0.0, 0.0, 1.0]]).numpy(), [0.0, 0.0, 0.0], atol=1e-5)), quaternion2euler([[0.0, 0.0, 0.0, 1.0]]).numpy()


q = [0.0, 0.0, 1.0, 0.0]
a1 = [np.pi, 0.0, 0.0]
a2 = [0.0, 0.0, np.pi]
assert np.all(np.isclose(quaternion2euler([q]).numpy(), a1, atol=1e-5)) or np.all(np.isclose(quaternion2euler([q]).numpy(), a2, atol=1e-5)), quaternion2euler([q]).numpy()

In [9]:
assert np.all(np.isclose(quaternion2euler([[0.0, 1.0, 0.0, 0.0]]).numpy(), [0.0, np.pi, 0.0], atol=1e-5)), quaternion2euler([[0.0, 1.0, 0.0, 0.0]]).numpy()
# assert np.all(np.isclose(quaternion2euler([[0.0, 0.0, 1.0, 0.0]]).numpy(), [0.0, 0.0, np.pi], atol=1e-5)), quaternion2euler([[0.0, 0.0, 1.0, 0.0]]).numpy()

q = [0.0, 0.0, 0.0, -1.0]
a1 = [np.pi, 0.0, np.pi]
a2 = [0.0, 0.0, 0.0]
assert np.all(np.isclose(quaternion2euler([q]).numpy(),a1, atol=1e-5)) or np.all(np.isclose(quaternion2euler([q]).numpy(), a2, atol=1e-5)), quaternion2euler([q]).numpy()

q = [0.0, 1.0, 0.0, 0.0]
a1 = [np.pi, np.pi, np.pi]
a2 = [0.0, np.pi, 0.0]
assert np.all(np.isclose(quaternion2euler([q]).numpy(), a1, atol=1e-5)) or np.all(np.isclose(quaternion2euler([q]).numpy(), a2, atol=1e-5)), quaternion2euler([q]).numpy()

In [10]:
e = [0.0, np.pi, 0.0]
q = euler2quaternion([e])
e_new = quaternion2euler([q])
e_new.numpy()

array([[[ 0.       ,  3.1415927, -0.       ]]], dtype=float32)

In [11]:
d_q(q, q).numpy()

array([7.88495335e-08])

In [12]:
def d_quat(quaternion1, quaternion2):
    dot_product = tf.reduce_sum(quaternion1 * quaternion2, axis=-1)
    # Ensure dot product is in range [-1. 1].
    eps = 8 * np.finfo(dot_product.dtype.as_numpy_dtype).eps
    dot_product *= (1.0 - eps)
    return 2.0 * tf.acos(tf.abs(dot_product))

In [13]:
d_quat(q, q).numpy()

array([0.00276214], dtype=float32)

# Additional tests

In [14]:
# def euler2quaternion(angles):
    
#     angles = tf.convert_to_tensor(value=angles)

#     shape.check_static(tensor=angles, tensor_name="angles", has_dim_equals=(-1, 3))
    
#     theta_z1, theta_y, theta_z0 = tf.unstack(angles, axis=-1)

#     # create rotation matrix
#     c1 = tf.cos(theta_z1)
#     c2 = tf.cos(theta_y)
#     c3 = tf.cos(theta_z0)

#     s1 = tf.sin(theta_z1)
#     s2 = tf.sin(theta_y)
#     s3 = tf.sin(theta_z0)

#     # PROJECTIONS CODE
#     r00 = c1*c2*c3-s1*s3
#     r01 = -(c1*s3+c2*c3*s1) ##
#     r02 = -(-c3*s2)  ##
#     r10 = -(-c3*s1-c1*c2*s3) ##
#     r11 = c1*c3-c2*s1*s3
#     r12 = s2*s3
#     r20 = -(c1*s2) ##
#     r21 = s1*s2 
#     r22 = c2
        
# #     print(r00.numpy(), r01.numpy(), r02.numpy())
# #     print(r10.numpy(), r11.numpy(), r12.numpy())
# #     print(r20.numpy(), r21.numpy(), r22.numpy())
# #     print("---")

#     w2 = 1/4*(1+ r00 + r11 + r22)
#     w2_is_pos = tf.greater(w2, 0)
    
#     x2 = -1/2*(r11+r22)
#     x2_is_pos = tf.greater(x2, 0)
    
#     y2 = 1/2*(1-r22)
#     y2_is_pos = tf.greater(y2, 0)
    
#     w = tf.compat.v1.where(w2_is_pos, tf.sqrt(w2), tf.zeros_like(w2))
#     x = tf.compat.v1.where(w2_is_pos, 1/(4*w)*(r21-r12),
#                                         tf.compat.v1.where(x2_is_pos, tf.sqrt(x2), tf.zeros_like(x2)))
#     y = tf.compat.v1.where(w2_is_pos, 1/(4*w)*(r02-r20),
#                                         tf.compat.v1.where(x2_is_pos, r01/(2*x), 
#                                                                     tf.compat.v1.where(y2_is_pos, tf.sqrt(y2), tf.zeros_like(y2))))
    
#     z = tf.compat.v1.where(w2_is_pos, 1/(4*w)*(r10-r01), 
#                                         tf.compat.v1.where(x2_is_pos, r02/(2*x), 
#                                                                     tf.compat.v1.where(y2_is_pos, r12/(2*y), tf.ones_like(y2))))
    
#     return tf.stack((x, y, z, w), axis=-1)

In [15]:
ap = [1.,2.,3.]
print(euler2quaternion([ap]).numpy()[0])
#print(euler2quaternion([ap], True).numpy()[0])
a = R.from_euler('zyz', ap)
print(a.as_quat())

[ 0.7080734  -0.4546487  -0.4912955   0.22484513]
[-0.70807342  0.45464871  0.4912955  -0.2248451 ]


In [16]:
ap = [-3., -2., 1.]
print(euler2quaternion([ap]).numpy()[0])
#print(euler2quaternion([ap], True).numpy()[0])
a = R.from_euler('zyz', ap)
print(a.as_quat())

[ 0.76514727  0.35017547 -0.4546487   0.29192662]
[ 0.7651474   0.35017549 -0.45464871  0.29192658]


In [17]:
ap = [ 0.14159265 , 2.00000001 ,-2.14159265]
print(euler2quaternion([ap]).numpy()[0])
#print(euler2quaternion([ap], True).numpy()[0])
a = R.from_euler('zyz', ap)
print(a.as_quat())

[ 0.7651474   0.35017544 -0.4546488   0.29192656]
[ 0.7651474   0.35017549 -0.45464871  0.29192658]


In [18]:
ap = [0.,0.,0.]
print(euler2quaternion([ap]).numpy()[0])
a = R.from_euler('zyz', ap)
print(a.as_quat())

[0. 0. 0. 1.]
[0. 0. 0. 1.]


In [19]:
ap = [0.,0.,np.pi]
print(euler2quaternion([ap]).numpy()[0])
a = R.from_euler('zyz', ap)
print(a.as_quat())

[0. 0. 1. 0.]
[0.000000e+00 0.000000e+00 1.000000e+00 6.123234e-17]


In [20]:
# def quaternion2euler(quaternions):
#     """https://github.com/tensorflow/graphics/blob/master/tensorflow_graphics/geometry/transformation/euler.py"""
    
#     def general_case(r02, r12, r20, r21, r22, eps_addition):
#         """Handles the general case."""
#         theta_y = tf.acos(r22)
#         #sign_sin_theta_y = safe_ops.nonzero_sign(tf.sin(theta_y))
        
#         r02 = safe_ops.nonzero_sign(r02) * eps_addition + r02
#         r22 = safe_ops.nonzero_sign(r22) * eps_addition + r22
        
#         theta_z0 = tf.atan2(r12, r02)
#         theta_z1 = tf.atan2(r21, -r20)
#         return tf.stack((theta_z1, theta_y, theta_z0), axis=-1)

#     def gimbal_lock(r22, r11, r10, eps_addition):
#         """Handles Gimbal locks.
#         It is gimbal when r22 is -1 or 1"""
#         sign_r22 = safe_ops.nonzero_sign(r22)
#         r11 = safe_ops.nonzero_sign(r11) * eps_addition + r11
        
#         theta_z0 = tf.atan2(sign_r22 * r10, r11)
        
#         theta_y = tf.constant(math.pi/2.0, dtype=r20.dtype) - sign_r22 * tf.constant(math.pi/2.0, dtype=r20.dtype)
#         theta_z1 = tf.zeros_like(theta_z0)
#         angles = tf.stack((theta_z1, theta_y, theta_z0), axis=-1)
#         return angles

#     with tf.compat.v1.name_scope(None, "euler_from_quaternion", [quaternions]):
#         quaternions = tf.convert_to_tensor(value=quaternions)

#         shape.check_static(
#             tensor=quaternions,
#             tensor_name="quaternions",
#             has_dim_equals=(-1, 4))

#         x, y, z, w = tf.unstack(quaternions, axis=-1)
#         tx = safe_ops.safe_shrink(2.0 * x, -2.0, 2.0, True)
#         ty = safe_ops.safe_shrink(2.0 * y, -2.0, 2.0, True)
#         tz = safe_ops.safe_shrink(2.0 * z, -2.0, 2.0, True)
#         twx = tx * w
#         twy = ty * w
#         twz = tz * w
#         txx = tx * x
#         txy = ty * x
#         txz = tz * x
#         tyy = ty * y
#         tyz = tz * y
#         tzz = tz * z

#         # The following is clipped due to numerical instabilities that can take some
#         # enties outside the [-1;1] range.
        
#         r00 = safe_ops.safe_shrink(1.0 - (tyy + tzz), -1.0, 1.0, True)
#         r01 = safe_ops.safe_shrink(txy - twz, -1.0, 1.0, True)
#         r02 = safe_ops.safe_shrink(txz + twy, -1.0, 1.0, True)

#         r10 = safe_ops.safe_shrink(txy + twz, -1.0, 1.0, True)
#         r11 = safe_ops.safe_shrink(1.0 - (txx + tzz), -1.0, 1.0, True)
#         r12 = safe_ops.safe_shrink(tyz - twx, -1.0, 1.0, True)

#         r20 = safe_ops.safe_shrink(txz - twy, -1.0, 1.0, True)
#         r21 = safe_ops.safe_shrink(tyz + twx, -1.0, 1.0, True)
#         r22 = safe_ops.safe_shrink(1.0 - (txx + tyy), -1.0, 1.0, True)
        
# #         print(r00.numpy(), r01.numpy(), r02.numpy())
# #         print(r10.numpy(), r11.numpy(), r12.numpy())
# #         print(r20.numpy(), r21.numpy(), r22.numpy())
# #         print("---")
        
#         eps_addition = asserts.select_eps_for_addition(quaternions.dtype)
#         general_solution = general_case(r02, r12, r20, r21, r22, eps_addition)
#         gimbal_solution = gimbal_lock(r22, r11, r10, eps_addition)
        
#         # The general solution is unstable close to the Gimbal lock, and the gimbal
#         # solution is not toooff in these cases.
#         # Check if r22 is 1 or -1
#         is_gimbal = tf.less(tf.abs(tf.abs(r22) - 1.0), 1.0e-6)
#         gimbal_mask = tf.stack((is_gimbal, is_gimbal, is_gimbal), axis=-1)
        
#         return tf.compat.v1.where(gimbal_mask, gimbal_solution, general_solution)    

In [21]:
# expected: [-3., -2., 1.]
qp = [ 0.7651474,   0.35017549 ,-0.45464871 , 0.29192658]
print(quaternion2euler([qp]).numpy()[0])
#print(euler2quaternion([ap], True).numpy()[0])
a = R.from_quat(qp)
print(a.as_matrix())
print(a.as_euler('zyz'))

[ 0.14159268  1.9999996  -2.141593  ]
[[ 0.34134335  0.80131982 -0.49129549]
 [ 0.27042365 -0.58431199 -0.7651474 ]
 [-0.90019763  0.12832006 -0.41614684]]
[ 0.14159265  2.00000001 -2.14159265]


In [22]:
#ap = [1.,2.,3.]
qp = [ 0.7080734 , -0.4546487 , -0.4912955  , 0.22484513]
print(quaternion2euler([qp]).numpy()[0])
#print(euler2quaternion([ap], True).numpy()[0])
a = R.from_quat(qp)
print(a.as_matrix())
print(a.as_euler('zyz'))

[1.        1.9999995 3.0000002]
[[ 0.10384657 -0.42291851 -0.90019766]
 [-0.86478012 -0.48547844  0.12832001]
 [-0.49129547  0.76514745 -0.41614679]]
[1.00000005 1.99999995 3.00000006]


In [24]:
ap = [1.,2.,3]
print(quaternion2euler(euler2quaternion([ap]).numpy()[0]))

tf.Tensor([1.        1.9999995 3.0000002], shape=(3,), dtype=float32)
